In [512]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

token = {
    '.' : 0, 
    ',' : 1, 
    '[' : 2, 
    ']' : 3, 
    '<' : 4, 
    '>' : 5, 
    '+' : 6, 
    '-' : 7,
    "START" : 8
    }

char = {
    0 : '.',
    1 : ',', 
    2 : '[', 
    3 : ']',  
    4 : '<',  
    5 : '>', 
    6 : '+',  
    7 : '-'
    # no START on purpose
    }

class BFgen(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, output_size, n_layers=2, batch_size=1):
        super(BFgen, self).__init__()
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.batch_size = batch_size
        
        self.encoder = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, n_layers, batch_first=True)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.softmax = nn.functional.softmax
        

    """
    forward
    Takes input_token and hidden memory state <- input to recursive layer
    returns output token and changed hidden memory state.
    """
    def forward(self, input_token):
        embeds = self.encoder(input_token)
        output, self.hidden = self.lstm(embeds, self.hidden)
        decoded = self.decoder(output)
        decoded = decoded.view(self.input_size, self.batch_size, self.output_size)
        probs = self.softmax(decoded[-1])
        return probs
    
    def init_hidden_zero(self):
        self.hidden = (Variable(torch.zeros(self.n_layers, self.batch_size, self.hidden_size)),
                      Variable(torch.zeros(self.n_layers, self.batch_size, self.hidden_size)),)
    
    def init_hidden_normal(self, variance=0.01):
        means = torch.zeros(self.n_layers, self.batch_size, self.hidden_size)
        std = torch.Tensor([variance]*self.hidden_size*self.n_layers*self.batch_size).unsqueeze(0)
        self.hidden = (Variable(torch.normal(means, std)), Variable(torch.normal(means, std)))





In [513]:
def token_to_tensor(input_token):
    tensor = torch.zeros(1, token_num).long()
    tensor[0][token[input_token]] = 1
    return tensor

def pred(sample):
    return char[np.argsort(sample)[-1]]

In [514]:
embedding_size = 10
hidden_size = 35
output_size = len(char.keys())
n_layers = 2
batch_size = 64

token_num = len(token.keys())

In [515]:
model = BFgen(token_num, embedding_size, hidden_size, output_size, n_layers, batch_size)

In [518]:
def evaluate(predict_len=1):
    input_token = "START"
    input_token = token_to_tensor(input_token)
    model.init_hidden_normal(variance=0.5)
    prediction = [""] * batch_size
    
    batched_input = torch.zeros((batch_size, token_num)).long()
    batched_input = batched_input + input_token
    batched_input = Variable(batched_input)
    
    for i in range(predict_len):
        output_token = model.forward(batched_input)
        batched_input = output_token
        next_tokens = np.apply_along_axis(pred, axis = 1, arr=output_token.data.numpy())
        for i in xrange(batch_size):
            prediction[i] += next_tokens[i] 
        
    return prediction

In [519]:
evaluate()

/Users/Wojtek/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ',',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 '<',
 '<',
 ',',
 ',',
 ']',
 ']',
 ']',
 ']',
 ']',
 '<',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 '<',
 '<',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ',',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 '<',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']',
 ']']